# Run the simulation

In the terminal: 

```bash
lmp < lammps_v3.in
```

# Analyze stream

In [ ]:
from imdclient.IMD import IMDReader
import MDAnalysis as mda
from MDAnalysis.analysis.rms import RMSF
from MDAnalysis.analysis.rdf import InterRDF
import concurrent.futures

u = mda.Universe("topology_after_min.data", "imd://localhost:8889", atom_style="id type x y z",)
g = u.atoms

rdf = InterRDF(g, g)
rdf._ts = u.trajectory.ts

rmsf = RMSF(g)
rmsf._ts = u.trajectory.ts

rdf._prepare()
rmsf._prepare()

executor = concurrent.futures.ThreadPoolExecutor()

count = 0
for ts in u.trajectory:
    rdf_frame = executor.submit(rdf._single_frame)

    rmsf._frame_index = count
    rmsf_frame =executor.submit(rmsf._single_frame)

    # Wait for each thread to finish
    rdf_frame.result()
    rmsf_frame.result()

    count += 1

rdf.n_frames = count
rdf._conclude()

rmsf._conclude()

rdf.results["rdf"]
rmsf.results["rmsf"]
